In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# 특정 워크플로우의 초기는 다음과 같이 생긴다
import uuid
from pathlib import Path
from typing import List, Dict
from datetime import datetime

from entity.process import GuardCondition, TaskSpec, TaskType, Layer, AgentNature, AgentRole, Process
from entity.validators import TokenValidator, SpecChainValidator
from entity.tokens import Token
from core.utils import load_resource_specs
from core.tokenDB import TokenRepository
import os
import logging
from core import logging_utils
from pathlib import Path
from core.tokenDB import TokenRepository
from core.utils import load_cfg

# 0. master config 불러오기
cfg = load_cfg(Path().parent.parent / "cfg" / "prod.yaml")

### 0. 노트북 작성목적

PROC_1 - 청킹 및 단순 적재 (Hard Process)
(금융연수원_소호_여신심사.md와 금융연수원_여신심사 및 관리.md)를 읽어들어, T1_LOAD, T1_SPLT, T1_TAG, T1_VECT의 태스크를 통해 "지식체계화"하는 것을 목표로 둔다. 즉, 최초로 형성되는 "비정형 지식 데이터베이스"의 자동화 프로세스 시도인 것이다.   
  
본 프로세스는 전부 비LLM함수로만 진행되는 단순 적재 Hard Process이다. 
지식체계화는 Proc_2, Proc_3, Proc_3에서 시도한다. 

내부 함수화, 즉 MCP_proxy는 스크립트 단위로 관리된다.  
현재 Hard Process를 불러오는 것도 없으니깐, 그것부터 정의를 한다. 

### 1. 최초 로드 및 DB 적재

### 2. ./Tools/tools:TokenizationPipeline 사용 예시

In [4]:
def load_markdown_files(data_path: str = "./data/input") -> dict[str, str]:
    """
    Update: 파일 경로가 들어오면 해당 파일만, 폴더가 들어오면 폴더 내 md 전체 로드
    """
    target_path = Path(data_path)
    
    # 1. 존재 여부 확인
    if not target_path.exists():
        print(f"[Error] 경로가 존재하지 않음: {data_path}")
        return {}

    # 2. Case A: 단일 파일인 경우
    if target_path.is_file():
        if target_path.suffix.lower() != '.md':
            print("[Warn] .md 파일이 아님")
            return {}
        return {target_path.stem: target_path.read_text(encoding="utf-8")}

    # 3. Case B: 디렉토리인 경우
    if target_path.is_dir():
        return {
            file.stem: file.read_text(encoding="utf-8")
            for file in target_path.glob("*.md")
        }

    return {}

class TokenizationPipeline:
    def __init__(self, db_repo: TokenRepository, chunk_size: int = 500, overlap: int = 50):
        self.repo = db_repo
        self.chunk_size = chunk_size
        self.overlap = overlap

    def _guess_topic(self, text: str) -> Dict[str, float]:
        """[Heuristic] 텍스트 키워드 기반 초기 Topic 추정 (LLM 전 단계)"""
        topics = {}

        # 향후 설정 필요
        if "매출" in text or "이익" in text: topics["TOPIC_FINANCE"] = 0.8
        if "담보" in text or "보증" in text: topics["TOPIC_COLLATERAL"] = 0.8
        if "소송" in text or "연체" in text: topics["TOPIC_RISK"] = 0.9
        
        if not topics: topics["TOPIC_GENERAL"] = 0.5
        return topics

    def _create_chunks(self, text: str) -> List[str]:
        """Fixed Size + Overlap 기반 청킹"""
        chunks = []
        start = 0
        text_len = len(text)
        
        while start < text_len:
            end = min(start + self.chunk_size, text_len)
            chunks.append(text[start:end])
            if end == text_len: break
            start += (self.chunk_size - self.overlap) # Overlap 이동
            
        return chunks

    def run(self, data_path: str):
        """Pipeline Execution: Load -> Split -> Tokenize -> Save"""
        # 1. Load
        raw_docs = load_markdown_files(data_path)
        if not raw_docs:
            print("[Info] 처리할 문서가 없습니다.")
            return

        total_tokens = 0
        
        for source_id, text in raw_docs.items():
            # 2. Split
            chunks = self._create_chunks(text)
            
            for idx, chunk_text in enumerate(chunks):
                # 3. Tokenize (Process Objectification)
                token = Token(
                    trace_id=f"TKN_{source_id}_{idx:04d}", # 순서 보장 ID
                    source_id=source_id,
                    history=["LOAD_SPLIT"],
                    content={
                        "text": chunk_text,
                        "chunk_index": idx,
                        "total_chunks": len(chunks),
                        "length": len(chunk_text),
                        "content_ref" : str(Path(data_path) / f"{source_id}.md")
                    },
                    topics=self._guess_topic(chunk_text)
                )
                
                # 4. Save
                self.repo.save(token)
                total_tokens += 1
                
        print(f"[Success] Pipeline Completed. Total Tokens Saved: {total_tokens}")

### 3. (Proc_1) MD 파일 토큰 형태로 DB 적재

In [5]:
repo = TokenRepository(cfg['location']['token_db'], table_name="SOHO_LOAN_raw_knowledge")
pipeline = TokenizationPipeline(repo, chunk_size=500, overlap=50)

In [6]:
pipeline.run("./data/input/금융연수원_소호 여신심사.md")

[Success] Pipeline Completed. Total Tokens Saved: 461


In [11]:
import sqlite3
import pandas as pd
import json
from pathlib import Path

def export_tokens_to_excel(db_path: str, output_path: str):
    if not Path(db_path).exists():
        print(f"[Error] DB file not found: {db_path}")
        return

    try:
        # 1. DB Connection
        conn = sqlite3.connect(db_path)
        
        # 2. Load Data to DataFrame
        query = "SELECT * FROM tokens ORDER BY source_id, trace_id"
        df = pd.read_sql_query(query, conn)
        
        # 3. Data Processing (Optional: JSON fields parsing)
        # 엑셀에서 보기 편하게 JSON 컬럼은 그대로 텍스트로 둠.
        
        # 4. Export to CSV (Excel 호환, BOM 추가)
        df.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"[Success] Exported {len(df)} rows to {output_path}")
        
    except Exception as e:
        print(f"[Fail] Export error: {e}")
    finally:
        conn.close()

if __name__ == "__main__":
    # DB 경로와 저장될 파일명 지정
    export_tokens_to_excel("./db/tb_cspn.db", "C:/Users/kakao/Desktop/temp_db/exported_tokens.csv")

[Success] Exported 1333 rows to C:/Users/kakao/Desktop/temp_db/exported_tokens.csv


### 4. (Proc_2) Knowledge Base 화

단순 메타데이터 필드를 정의해주는 것으로는 끝나선 안된다. 우리가 이 문서를 인식시킴으로써 "무얼 성취하고자 하는가"가 본 POC에서 중요하다.  
앞서 Semantic_Layer_Seed에서 이야기했던 개념, 우선 손으로 다시 써서 익게 하자.  

프로세스에는 Hard / Soft (Context-rich)~(Newborn) / Liquid이 있다. 즉, 우리는 프로세스의 정형화 정도에 따라 
프로세스를 분류하고 있는 것이다. 또한, 우리가 읽어들인 것은 결국 절차의 집합소이며, 상당부분은 Liquid일 것이다.  

왜 애초에 프로세스를 언어화 했는가?  

1. 토큰 기반 통제 가능성이다.  
- KPMG가 내리는 전문가의 정의는 "충분한 데이터가 제공됐을때, 해당 데이터를 전부 고려하여 정해진 절차에 따라 판단을 내릴 수 있는 agent"이다. 
- 토큰 기반 invoke는 데이터가 없으면 멈춘다. 사용자에게 요구한다. 환각을 방지한다. 

2. 추적 가능하다. 모든 세션과 비정형데이터는 토큰이라는 공통 언어로 통제된다. 

3. 지식이 진화한다. 
- 각 토큰은 배치를 통해 "self-study"되며, 맥락이 풍부해진다. (smart object화)





#### 4.1. LLM 메서드 필요 (agents.py)

### 4.2. 토큰 업데이트 및 Merge 로직 필요